<a href="https://colab.research.google.com/github/sakuna47/Spam_Detection/blob/SD_BackEnd/Spam_Detection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [42]:
!pip install pandas numpy scikit-learn nltk flask streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.4 MB/s eta 0:00:00


In [56]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
import pandas as pd

# Load the dataset with low_memory=False to prevent mixed type warnings
file_path = "/content/drive/My Drive/Spam Detection Web App/sms_scam_detection_dataset_merged_with_lang.csv"
df = pd.read_csv(file_path, encoding="latin-1", low_memory=False)


# Keep only the relevant columns and rename them
df = df[["label", "text"]]
df.columns = ["label", "message"]

# Normalize label case to fix "Spam" vs "spam"
df["label"] = df["label"].str.lower()


# Explore the dataset
print(df.head())
print(df["label"].value_counts())


# Check dataset shape
print(f"Dataset loaded with {df.shape[0]} rows and {df.shape[1]} columns")

# Print column names to verify correct ones exist
print("Column names in dataset:", df.columns)


  label                                            message
0   ham  Your opinion about me? 1. Over 2. Jada 3. Kusr...
1   ham  What's up? Do you want me to come online? If y...
2   ham                       So u workin overtime nigpun?
3   ham  Also sir, i sent you an email about how to log...
4  spam  Please Stay At Home. To encourage the notion o...
label
ham     78321
spam    60492
Name: count, dtype: int64
Dataset loaded with 138813 rows and 2 columns
Column names in dataset: Index(['label', 'message'], dtype='object')


In [75]:
print("Unique labels before mapping:", df["label"].unique())
df["label"] = df["label"].map({"ham": 0, "spam": 1})

# Drop NaN values after mapping
df = df.dropna(subset=["label"])
print("Unique labels after mapping:", df["label"].unique())

# Preprocess the Data
# Convert labels to binary values (ham = 0, spam = 1)
df["label"] = df["label"].map({"ham": 0, "spam": 1})

# Download NLTK stopwords
nltk.download("stopwords")

# Text Preprocessing Function
def preprocess_text(text):
    if pd.isna(text):  # Handle missing messages
        return ""
    text = text.lower()  # Convert to lowercase
    stop_words = set(stopwords.words("english"))  # Load stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    text = "".join([char for char in text if char.isalpha() or char == " "])  # Remove special characters/numbers
    return text

# Apply preprocessing
df["message"] = df["message"].apply(preprocess_text)

# Print final dataset info
print(f"Total dataset size after cleaning: {len(df)}")

Unique labels before mapping: ['ham' 'spam']
Unique labels after mapping: [0 1]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Total dataset size after cleaning: 138813


In [69]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df["message"], df["label"], test_size=0.2, random_state=42)

In [70]:
# Feature Extraction
# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_tfidf = vectorizer.transform(X_test)

In [72]:
print("Original unique labels in 'label' column:", df['label'].unique())

print(f"NaN values in target variable (y_train): {y_train.isna().sum()}")
print("NaN values in 'label' column before mapping:", df['label'].isna().sum())
print("Unique labels after mapping:", df["label"].unique())
print(f"NaN values in 'label' column after cleanup: {df['label'].isna().sum()}")

df = df.dropna(subset=["label"])
print(f"Rows after dropping NaN labels: {df.shape[0]}")

# Initialize the Naive Bayes model
model = MultinomialNB()

# Train the model
model.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Original unique labels in 'label' column: []
NaN values in target variable (y_train): 111050
NaN values in 'label' column before mapping: 0
Unique labels after mapping: []
NaN values in 'label' column after cleanup: 0
Rows after dropping NaN labels: 0


ValueError: Input y contains NaN.

In [ ]:
# Define the path on Google Drive
model_path = "/content/drive/My Drive/Spam Detection Web App/spam_model.pkl"
vectorizer_path = "/content/drive/My Drive/Spam Detection Web App/tfidf_vectorizer.pkl"

# Save the model
with open(model_path, "wb") as f:
    pickle.dump(model, f)

# Save the vectorizer
with open(vectorizer_path, "wb") as f:
    pickle.dump(vectorizer, f)
